In [2]:
import pandas as pd
import os

from kiwipiepy import Kiwi

from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from bertopic.representation import KeyBERTInspired

In [3]:
IN_DATA_PATH = './data/01_out'
OUT_DATA_PATH = './data/02_out'

in_data_files = []
in_docs = []
total_docs = []

# 폴더내 파일리스트를 가져온다.
input_files = os.listdir(IN_DATA_PATH)

# 입력 파일 로딩 RAW_DATA_PATH에 있는 파일을 DataFrame으로 읽어 드린다.
for in_file in input_files:
    print ("Loading file :%s ..."%in_file)
    df = pd.read_csv(os.path.join(IN_DATA_PATH,in_file), encoding='utf-8-sig')
    docs = df['generated_text'].tolist()
    in_data_files.append(df)
    in_docs.append(docs)
    total_docs.extend(docs)    


Loading file :umsun_A_2_sborder.csv ...
Loading file :umsun_B_sborder.csv ...
Loading file :umsun_clean_C.csv ...


In [4]:
# 불용어를 정의한다
user_stop_word = [ "거", "바", "뻥", "눌", ]

# 토크나이저에 ***만 추가한다      https://incredible.ai/nlp/2016/12/28/NLP/
extract_pos_list = ["NNG", "NNP", "NNB", "NR", "NP", "VV", "VA","MAG","IC"]

class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, text):
        result = list()
        for word in self.tagger.tokenize(text):
            # 명사이고, 길이가 2이상인 단어이고, 불용어 리스트에 없으면 추가하기
            if word[1] in extract_pos_list and word[0] not in user_stop_word: #len(word[0]) > 1 and
                result.append(word[0])
        return result

# 커스텀 단어 등록
kiwi = Kiwi()
kiwi.add_user_word("노브랜드","NNP")
kiwi.add_user_word("한살림","NNP")
kiwi.add_user_word("이마트","NNP")
kiwi.add_user_word("피코크","NNP")
kiwi.add_user_word("누네띠네","NNP")
kiwi.add_user_word("동원참치","NNP")
kiwi.add_user_word("떡뻥","NNP")
kiwi.add_user_word("베지밀","NNP")
kiwi.add_user_word("네스카페","NNP")
kiwi.add_user_word("맥스봉","NNP")
kiwi.add_user_word("오뚜기","NNP")
kiwi.add_user_word("오징어집","NNP")
kiwi.add_user_word("오곡쿠키","NNP")

kiwi.add_user_word("10대","NR")
kiwi.add_user_word("20대","NR")
kiwi.add_user_word("30대","NR")
kiwi.add_user_word("40대","NR")
kiwi.add_user_word("50대","NR")
kiwi.add_user_word("베스킨라빈스","NNP")
kiwi.add_user_word("이산화규소","NNP")
kiwi.add_user_word("중위험","NNP")
kiwi.add_user_word("고위험","NNP")
kiwi.add_user_word("저위험","NNP")
kiwi.add_user_word("우리밀","NNP")
kiwi.add_user_word("열일","NNG")
kiwi.add_user_word("유전자변형","NNG")
kiwi.add_user_word("유화증진제","NNG")
kiwi.add_user_word("유화제","NNG")
kiwi.add_user_word("스프레드","NNG")
kiwi.add_user_word("식품첨가물","NNG")



True

In [6]:
custom_tokenizer = CustomTokenizer(kiwi)
vectorizer = CountVectorizer(tokenizer=custom_tokenizer)

In [7]:
# Pre-calculate embeddings

#embedding_model = SentenceTransformer("sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")

embedding_model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")
embeddings = embedding_model.encode(total_docs, show_progress_bar=True)

Batches:   0%|          | 0/625 [00:00<?, ?it/s]

In [8]:
# 차원축소
hdbscan_model = HDBSCAN(min_cluster_size=20, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

In [9]:
# zero-shot
ko_classifier = pipeline(
    task='zero-shot-classification',
    model='MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli',
    device=0,
    # hypothesis_template='구매하는 이유는 {} 이다.',
)


In [10]:
CANDIDATE_LABELS = ['첨가물','성분','위험','아기','안심','칼로리','중위험','아이']

In [11]:
# [0] => A파일 , [1] => B파일 , [2] => C파일

docs = in_docs[2]

In [12]:
topic_model = BERTopic(
        embedding_model=embedding_model,
        hdbscan_model=hdbscan_model,
        vectorizer_model=vectorizer,
        nr_topics="auto", # 문서를 대표하는 토픽의 갯수
        # top_n_words=4,
        zeroshot_topic_list=CANDIDATE_LABELS,
        zeroshot_min_similarity=.5,
        # representation_model=ko_classifier,
        representation_model=KeyBERTInspired(),
        calculate_probabilities=True
)
	
topics, probs = topic_model.fit_transform(docs)
topic_model.get_topic_info()[:30]

/home/bookendus/.local/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


,Topic,Count,Name,Representation,Representative_Docs
0,0,1003,첨가물,"[첨가물, 첨가, 맛있, 설탕, 영양, 우유, 성분, 유기농, 나트륨, 맛]","[첨가물이 없어서 좋아요., 첨가물도 없고 고소해서 좋아해요., 첨가물이 없어서 좋..."
1,1,624,성분,"[성분, 크림, 감미료, 첨가물, 젤리, 천연, 함유, 향료, 초콜릿, 맛있]","[성분이 좋네요., 맛도 좋고 한데 성분 아쉽네요., 맛도 성분도 좋아요.]"
2,2,135,아기,"[아기, 아가, 애기, 아이, 임신, 맛있, 우유, 유기농, 기쁘, 요구르트]","[아기한테 먹이면 좋겠어요., 아기가 좋아해요., 아기가 좋아해요.]"
3,3,125,안심,"[괜찮, 안심, 만족, 무난하, 꿀맛, 최고, 다행, 인정, 건강, 감사]","[괜찮아 괜찮아., 괜찮아요., 괜찮은 제품.]"
4,4,38,중위험,"[중위험, 저위험, 고위험, 대비, 안, 가성비, 하나, 아커, 필요, 대박]","[중위험., 중위험., 중위험이 아쉽네요.]"
5,5,32,칼로리,"[칼로리, 섭취, 나트륨, 다이어트, 감미료, 함량, 다이어터, 지방산, 요구르트,...","[칼로리가 낮아서 좋다며 먹었는데 감미료가., 칼로리가 너무 높다고 생각이 듭니다...."
6,6,22,아이,"[아이, 아가, 아기, 너무, 맛있, 울아, 어머, 울, 착하, 성분]","[제가 좋아해요 아이는 주지 말아야겠어요., 아이가 좋아해요., 아이가 좋아해요.]"
7,7,4,위험,"[위험, 저위험, 중위험, 하하, 아, 하나, 분, 조금, 있, 유명]","[위험분이 있군요., 위험분이 하나 있네요 하하., 거버 유명한 회산데 중위험군 조..."
8,8,3151,-1_젤리_단맛_맛있_간식,"[젤리, 단맛, 맛있, 간식, 맛나, 첨가물, 식감, 맛, 식품, 먹]",[밥하기 싫을 때 간편하게 먹을 수 있어서 좋은데 첨가물도 없다니 더 자주 먹을 것...
9,9,1133,0_맛있_간식_맛_식품,"[맛있, 간식, 맛, 식품, 첨가물, 먹, 초콜릿, 성분, 달달하, 식감]","[아이들과 먹기 좋아서., 식품 첨가물에 들어가는 산도 조절제에는 안전한 성분도 있..."


In [ ]:
hierarchical_topics = topic_model.hierarchical_topics(docs)
tree = topic_model.get_topic_tree(hierarchical_topics)
print(tree)

In [ ]:
# Coherence
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel

# Preprocess documents for coherence evaluation
documents = pd.DataFrame({"Document": docs, "ID": range(len(docs)), "Topic": topics})
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ''.join})
cleaned_docs = topic_model._preprocess_text(documents_per_topic.Document.values)

# Extract vectorizer and analyzer for BERTopic
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()

# Extract features for topic coherence evaluation
words = vectorizer.get_feature_names_out()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]

# Extract words in each topic if they are non-empty and exist in the dictionary
topic_words = []
#topic_words = [list(zip(*model_2024_B_3.get_topic(topic)))[0] for topic in range(len(set(topics))-1)]
for topic in range(len(set(topics))-topic_model._outliers):
  words = list(zip(*topic_model.get_topic(topic)))[0]
  words = [word for word in words if word in dictionary.token2id]
  topic_words.append(words)
topic_words = [words for words in topic_words if len(words)>0]

# Evaluate Coherence
coherence_model = CoherenceModel(topics=topic_words, texts=tokens, corpus=corpus, dictionary=dictionary, coherence='c_v')
coherence = coherence_model.get_coherence()
print("Coherence Score: ", coherence)